# Counter-Current Heat Exchanger

Simulating the startup transient of a counter-current shell-and-tube heat exchanger, showing how temperature profiles develop along the exchanger length.

The exchanger is discretized into $N$ cells along its length. In each cell, the hot and cold streams exchange heat proportional to the local temperature difference:

$$\frac{dT_{h,i}}{dt} = \frac{F_h}{V_\text{cell,h}} (T_{h,i-1} - T_{h,i}) - \frac{UA_\text{cell}}{\rho_h C_{p,h} V_\text{cell,h}} (T_{h,i} - T_{c,i})$$

$$\frac{dT_{c,i}}{dt} = \frac{F_c}{V_\text{cell,c}} (T_{c,i+1} - T_{c,i}) + \frac{UA_\text{cell}}{\rho_c C_{p,c} V_\text{cell,c}} (T_{h,i} - T_{c,i})$$

The hot stream flows from cell 1 to $N$, the cold stream flows counter-currently from cell $N$ to 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Scope

from pathsim_chem.process import HeatExchanger

Configure a heat exchanger with 10 cells. Both sides start at 300 K (cold), then hot fluid at 370 K enters. This lets us watch the temperature profile build up from the initial uniform state. The high $UA$ relative to the flow rate ensures significant heat transfer.

In [ ]:
hx = HeatExchanger(
    N_cells=10,        # spatial discretization
    F_h=0.002,         # hot flow rate [m³/s]
    F_c=0.002,         # cold flow rate [m³/s]
    V_h=0.1,           # hot-side volume [m³]
    V_c=0.1,           # cold-side volume [m³]
    UA=5000.0,         # overall heat transfer [W/K]
    rho_h=1000.0,      # hot-side density [kg/m³]
    Cp_h=4184.0,       # hot-side heat capacity [J/(kg·K)]
    rho_c=1000.0,      # cold-side density [kg/m³]
    Cp_c=4184.0,       # cold-side heat capacity [J/(kg·K)]
    T_h0=300.0,        # initial hot-side temp [K]
    T_c0=300.0,        # initial cold-side temp [K]
)

Feed hot water at 370 K and cold water at 290 K. Record the outlet temperatures over time.

In [ ]:
T_h_in = Source(func=lambda t: 370.0)   # hot inlet [K]
T_c_in = Source(func=lambda t: 290.0)   # cold inlet [K]

scp = Scope(labels=["T_h_out [K]", "T_c_out [K]"])

sim = Simulation(
    blocks=[T_h_in, T_c_in, hx, scp],
    connections=[
        Connection(T_h_in, hx),          # T_h_in -> port 0
        Connection(T_c_in, hx[1]),        # T_c_in -> port 1
        Connection(hx, scp),              # T_h_out -> scope port 0
        Connection(hx[1], scp[1]),        # T_c_out -> scope port 1
    ],
    dt=0.02,
)

sim.run(200)

In [ ]:
time, signals = scp.read()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Outlet temperatures over time
ax1.plot(time, signals[0], label="Hot outlet")
ax1.plot(time, signals[1], label="Cold outlet")
ax1.axhline(370, color="gray", ls="--", alpha=0.4, label="Hot inlet")
ax1.axhline(290, color="gray", ls="-.", alpha=0.4, label="Cold inlet")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Temperature [K]")
ax1.set_title("Outlet Temperatures")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Steady-state temperature profile along the exchanger
state = hx.engine.x0
T_h_profile = state[0::2]
T_c_profile = state[1::2]
cells = np.arange(1, len(T_h_profile) + 1)

ax2.plot(cells, T_h_profile, "o-", label="Hot stream")
ax2.plot(cells, T_c_profile, "s-", label="Cold stream")
ax2.set_xlabel("Cell number")
ax2.set_ylabel("Temperature [K]")
ax2.set_title("Temperature Profile (Steady State)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

At startup, both sides are cold. The hot fluid quickly warms the exchanger from the inlet side while the cold fluid absorbs heat. At steady state, the counter-current arrangement creates the characteristic temperature cross — the cold outlet approaches the hot inlet and vice versa, achieving high thermal efficiency.